# IMPORTATIONS

## LINK GOOGLE DRIVE AND GOOGLE COLAB

In [0]:
# Exécutez cette cellule pour installer votre Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## LIBRARY IMPORTATIONS

In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torchsummary import summary
from collections import OrderedDict

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


# VARIABLES

In [0]:
# Root directory for dataset
dataroot = '/content/drive/My Drive/Colab Notebooks/Cat_dataset'
 

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 32

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 2000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# LOAD AND TEST THE DATASET

In [0]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

RuntimeError: ignored

# MODEL

## WEIGHTS

In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## GENERATOR

### CREATION

In [0]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        
        layers = OrderedDict()
        output_size = image_size * 8
        i_layer = 0
        
        while(output_size >= ngf):
            if i_layer == 0:
                layers[str(i_layer)] = nn.ConvTranspose2d(nz, output_size, 4, 1, 0, bias=False)
            else:
                layers[str(i_layer)] = nn.ConvTranspose2d(output_size * 2, output_size, 4, 2, 1, bias=False)
          
            layers[str(i_layer + 1)] = nn.BatchNorm2d(output_size)
            layers[str(i_layer + 2)] = nn.ReLU(True)
            
            if output_size == ngf:
                layers[str(i_layer + 3)] = nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)
                layers[str(i_layer + 4)] = nn.Tanh()
            output_size = output_size // 2
            i_layer += 3
        
          
        str_main = "nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),\
            nn.BatchNorm2d(ngf * 8),\
            nn.ReLU(True),\
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),\
            nn.BatchNorm2d(ngf * 4),\
            nn.ReLU(True),\
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),\
            nn.BatchNorm2d(ngf * 2),\
            nn.ReLU(True),\
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),\
            nn.BatchNorm2d(ngf),\
            nn.ReLU(True),\
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),\
            nn.Tanh()"
        self.main = nn.Sequential(layers)

    def forward(self, input):
        return self.main(input)

### INSTANTIATION

In [0]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)
summary(netG, (100, 1, 1))

## DISCRIMINATOR

### CREATION

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        
        layers = OrderedDict()
        output_size = image_size
        i_layer = 0
        
        padding  = 0
        
        while(output_size <= image_size * 8):
            
            if i_layer == 0:
                layers[str(i_layer)] = nn.Conv2d(nc, output_size, 4, 2, 1, bias=False)
                layers[str(i_layer + 1)] = nn.LeakyReLU(0.2, inplace=True)
                i_layer -= 1
            else:
                layers[str(i_layer)] = nn.Conv2d(output_size // 2, output_size, 4, 2, 1, bias=False)
                layers[str(i_layer + 1)] = nn.BatchNorm2d(output_size)
                layers[str(i_layer + 2)] = nn.LeakyReLU(0.2, inplace=True)
          
            
            if output_size == image_size * 8:
                i_layer += 3
                n_stride = np.log(image_size / ndf) / np.log(2)
                print(n_stride)
                """while n_stride > 0:
                    layers[str(i_layer)] = nn.Conv2d(output_size, output_size, 4, 2, padding, bias=False)
                    layers[str(i_layer + 1)] = nn.BatchNorm2d(output_size)
                    layers[str(i_layer + 2)] = nn.LeakyReLU(0.2, inplace=True)
                    n_stride -= 1
                    i_layer += 3"""
                layers[str(i_layer)] = nn.Conv2d(output_size, 1, 4, 1, 0, bias=False)
                layers[str(i_layer + 1)] = nn.AdaptiveAvgPool2d(1)
                layers[str(i_layer + 3)] = nn.Sigmoid()
                
          

            output_size = output_size * 2
            i_layer += 3
        
        
        self.main = nn.Sequential(layers)
        """# input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()"""

    def forward(self, input):
        return self.main(input)

### INSTANTIATION

In [0]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)
summary(netD, (3, image_size, image_size))

"""
Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

"""

## COMPILATIONS

In [0]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# AUXILIARY

## SAVE AND LOAD

In [0]:
save_dir = os.path.join(dataroot, "model")  
def save():
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    torch.save(netG.state_dict(), os.path.join(save_dir, 'model' + '_G.pkl'))
    torch.save(netD.state_dict(), os.path.join(save_dir, 'model' + '_D.pkl'))
    print('model saved...')    

def load():
    if os.path.exists(save_dir):
        netG.load_state_dict(torch.load(os.path.join(save_dir, 'model' + '_G.pkl')))
        netD.load_state_dict(torch.load(os.path.join(save_dir, 'model' + '_D.pkl')))
        print('model loaded...')
    else:
      print('no model founded...')

## DISPLAY

In [0]:
def display():
    # Grab a batch of real images from the dataloader
    real_batch = next(iter(dataloader))

    # Plot the real images
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.axis("off")
    plt.title("Real Images")
    plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

    # Plot the fake images from the last epoch
    plt.subplot(1,2,2)
    plt.axis("off")
    plt.title("Fake Images")
    plt.imshow(np.transpose(img_list, (1,2,0)))
    plt.show()


# TRAINING

In [0]:
# Training Loop

# Lists to keep track of progress
img_list = []

iters = 0
load()
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 1 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if iters % 10 == 0:
            save()
            
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 100 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list = vutils.make_grid(fake, padding=2, normalize=True)
            display()
            

        iters += 1
